<a href="https://colab.research.google.com/github/GrzegorzAndrzejczak/Programowanie-w-Pythonie/blob/main/CASE-lighthouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Dane: lighthouse.csv